# Imports - TO DO add helpers, streamline process

In [1]:
import pandas as pd
import os
import re
import numpy as np
import ast
import matplotlib.pyplot as plt


# Helper Functions

In [2]:
def get_attack_spacing_intermittent(intermittent_df): # Calculate distance between each attack and add to Dataframe
    attack_spacing = []

    for simulation_instance in range(len(intermittent_df)):
        diff_1 = intermittent_df["point.params.random_t2"][simulation_instance] - intermittent_df["point.params.random_t1"][simulation_instance] # t2 > t1
        diff_2 = intermittent_df["point.params.random_t3"][simulation_instance] - intermittent_df["point.params.random_t2"][simulation_instance]
        diff_3 = intermittent_df["point.params.random_t4"][simulation_instance] - intermittent_df["point.params.random_t3"][simulation_instance]

        sim_gaps = np.array([diff_1,diff_2,diff_3])
        attack_spacing.append(sim_gaps)

    attack_spacing = np.array(attack_spacing)
    col_names = ["t1_t2", "t2_t3", "t3_t4"]
    time_df = pd.DataFrame(attack_spacing, columns=col_names)

    return(pd.concat([intermittent_df, time_df],axis=1))



def velocity_at_attack_intermittent(velocities_df, intermittent_df): # Collect the attackers speed at each attack and append to Dataframe
    simulation_velocities_arr = []
    velocities_df = velocities_df.reset_index(drop=True)
    
    for j in range(0,len(velocities_df)):
        sim_velocities = []
        for i in velocities_df.columns:
            vals = ast.literal_eval(velocities_df[i][j])
            velocities = [i[1] for i in vals]
            sim_velocities.append(velocities)
        
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance

    simulation_velocities_arr = np.array(simulation_velocities_arr)

    
    col_names = ["t1_velocity", "t2_velocity", "t3_velocity", "t4_velocity"]
    velocity_df = pd.DataFrame(simulation_velocities_arr, columns=col_names)

    return(pd.concat([intermittent_df, velocity_df],axis=1))

# TODO FIX THIS FUNC
def velocity_at_attack_persistent(velocities_df, persistent_df):
    simulation_velocities_arr = []
    velocities_df = velocities_df.reset_index(drop=True)
    
    for j in range(0,len(velocities_df)):
        sim_velocities = []
        for i in velocities_df.columns:
            vals = ast.literal_eval(velocities_df[i][j])
            velocities = [i[1] for i in vals]
            sim_velocities.append(velocities)
        
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance

    simulation_velocities_arr = np.array(simulation_velocities_arr)
                                         


# Visualization Tools

In [3]:
def generate_persistent_attack_graphs(sim_velocities,att_freq,att_amplitude):
    for i in range(5):
        plt.figure()    

        x = range(len(sim_velocities[i][0]))
        plt.plot(x,sim_velocities[i][0])

        plt.title(f"Velocity Over time with attack Frequency {int(att_freq[i])} and amplitude {att_amplitude[i]:.2g}")

        plt.xlabel("Timestep (per .1 seconds)")
        plt.ylabel(f"Velocity")

        plt.show()

#def visualize_intermittent_velocities(sim_velocities,)



# Classify File Instances

In [5]:
file_path = "../outputs/persistent_ce/persistent_attack_ce.csv"
dir_path = "../outputs/persistent_ce"

def classify_simulations(files):
    ce_reg = r'.*\d_cex.*'
    nc_reg = r'.*no_cex.*'
    att_reg = r".*attacker_crash.*"

    reg = [ce_reg,nc_reg,att_reg]
    classified_cases = [[],[],[]]
    for simulation in files:
        for i in range(len(reg)):
            found = re.match(reg[i],simulation)
            if found:
                classified_cases[i].append(simulation)
                break
    return classified_cases

def read_folder(folder,sampler_file, attack_type, velocity_file):    
    files = os.listdir(folder)
    final = classify_simulations(files)

    sum = 0
    for i in final:
        sum += len(i)
        print(len(i))

    #sanity check
    print(sum == len(files)-2) #account for the velocities and sampler files 

    simulation_params_df = pd.read_csv(sampler_file)

    if attack_type == "int":
        return(preprocess_intermittent(final,simulation_params_df,velocity_file))
    if attack_type == "per":
        return(preprocess_persistent(final,simulation_params_df,velocity_file))
    else:
        print("Error classifying attack type")


def preprocess_intermittent(classified_files, sampler_data,velocity_file):
    intermittent_cols = ["point.params.random_thr_br1","point.params.random_thr_br2","point.params.inter_vehicle_distance", "point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4","point.params.random_thr_br1","rho"]
    sampler_data =sampler_data[intermittent_cols]
    output = get_attack_spacing_intermittent(sampler_data)
    output = velocity_at_attack_intermittent(sampler_data,velocity_file)

    return output


def preprocess_persistent(classified_files,sampler_data):
    pass


# Curr work

Persistent_attack uses frequency to define how often the attack is applied. What are some interesting feature we can extract from this? 
 Total attacks?
 Time between attacks?
 Velocity at attack time would still be interesting just a bit more complex to extract.


Likely need to rerun data as there was a useless variable EGO_BRAKING_THESHOLD being included in the search space.. I am not sure how much of an effect this would have had but can always re run and see what happens

In [6]:
file_path = "../outputs/persistent_ce/persistent_attack_ce.csv"
dir_path = "../outputs/persistent_ce"

simulation_params_df = pd.read_csv("../outputs/persistent_ce/persistent_attack_ce.csv")

In [7]:
persistent_cols = [ "point.params.amplitude_acc", "point.params.attack_time", "point.params.frequency", "point.params.inter_vehicle_distance", "rho"]
intemittent_cols = ["point.params.random_thr_br1","point.params.random_thr_br2","point.params.inter_vehicle_distance", "point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4","point.params.random_thr_br1","rho"]

In [22]:
evaluation_simulation_params_df = simulation_params_df[persistent_cols]
print(len(evaluation_simulation_params_df))

500


In [12]:
files = os.listdir(dir_path)
classified_files = classify_simulations(files)

In [9]:
evaluation_simulation_params_df.describe()

,point.params.amplitude_acc,point.params.attack_time,point.params.frequency,point.params.inter_vehicle_distance,rho
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.341717,6.507021,6.109624,11.331299,2.282775
std,0.274452,1.975794,2.955776,4.929331,4.627224
min,0.001723,0.036584,0.057499,7.008833,-0.273221
25%,0.124351,6.188735,3.267220,8.425303,0.066527
50%,0.265883,6.950120,6.355811,9.749395,0.242624
75%,0.534049,7.575427,9.023248,11.140633,1.478580
max,0.994191,9.980661,9.993043,29.065113,23.108299


In [42]:
vel_path = "../outputs/persistent_ce/persistent_attack.scenic500_velocities.csv"
vel_df = pd.read_csv(vel_path)

# Constructing features for the persistent Attack

In [ ]:
simulation_velocities_arr = [] # process velocity values
velocities_df = vel_df.reset_index(drop=True)

for j in range(0,len(vel_df)):
    sim_velocities = []
    for i in vel_df.columns:
        vals = ast.literal_eval(vel_df[i][j])
        velocities = [i[1] for i in vals]
        sim_velocities.append(velocities)
    
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance

simulation_velocities_arr = np.array(simulation_velocities_arr)


velocity_at_attack = []  # construct custom features
avg_speed_after_attck = []
for i in range(len(attack_freq)):
    j = 0
    attack_span = int(attack_freq[i] * 10)
    velocity_at_attack.append(np.average(simulation_velocities_arr[i][0][attack_span-2:attack_span+2])) # average out over a few steps
    avg_speed_after_attck.append(np.average(simulation_velocities_arr[i][0][attack_span:]))

In [ ]:
attack_freq = evaluation_simulation_params_df["point.params.frequency"]
attack_amplitude = evaluation_simulation_params_df["point.params.amplitude_acc"]


6.620580152379367


In [ ]:
generate_persistnet_attack_graphs(simulation_velocities_arr,attack_freq,attack_amplitude)

In [66]:
print(len(simulation_velocities_arr[0][2]))

252


In [100]:

print(len(simulation_velocities_arr[499][0]))

252


In [108]:
print(velocity_at_attack)
print(avg_speed_after_attck)

[5.7734720750794, 5.950169396784643, 9.339815147170405, 0.3595771802901905, 14.045866491086969, 12.166850806591087, 2.5417253469453787, 14.781004906775376, 0.00016132388374537297, 9.799123118067937, 10.871996405423296, 9.799123118067937, 8.606800797270415, 3.9909225115645652, 0.9506238654068938, 0.00013629598867290392, 2.1424475733057, 9.799123118067937, 11.895731457252197, 12.033711913865048, 9.131713752065291, 1.390599717590953, 0.0003172060607107986, 6.233650558945429, 7.3318810496401055, 5.950169396784643, 12.294538259745252, 0.08051835488123456, 2.1897444598701012, 0.00855756588877622, 6.445665382774046, 0.1548511264236491, 12.867447140117534, 3.058661784079267, 13.508861782016641, 6.7673045088063875, 5.006859693035945, 0.02744961253581954, 4.041763189794045, 0.5942118064777997, 7.6318659874494585, 8.142252443268625, 9.799123118067937, 10.091099502564848, 5.433573456938246, 11.207920075243369, 0.31089683004252866, 6.776151993856992, 9.799123118067937, 6.502313504394083, 9.13171375

# Isolate different crash instances

In [17]:
evaluation_simulation_params_ce_df = evaluation_simulation_params_df[:len(classified_files[0])]
non_ce_idx = len(classified_files[1]) + len(classified_files[2])
evaluation_simulation_params_nce_df = evaluation_simulation_params_df[:non_ce_idx]

# So I thought it might be the case that the cross entropy sampler go this many results simply becuase it was concentrating at a certain set distance apart and this definitely seems to support that motion!

In [18]:
evaluation_simulation_params_ce_df.describe()

,point.params.amplitude_acc,point.params.attack_time,point.params.frequency,point.params.inter_vehicle_distance,rho
count,122.000000,122.000000,122.000000,122.000000,122.000000
mean,0.231911,6.936071,6.764863,9.918378,-0.219390
std,0.217093,1.047438,2.832357,0.862728,0.049373
min,0.004958,0.254307,2.094187,7.128460,-0.273221
25%,0.100293,6.382794,3.637671,9.391736,-0.243762
50%,0.155324,7.022333,8.320250,10.011812,-0.238714
75%,0.297434,7.436284,9.147684,10.558582,-0.218363
max,0.968585,9.875557,9.917750,11.504694,-0.037455


In [19]:
evaluation_simulation_params_nce_df.describe()

,point.params.amplitude_acc,point.params.attack_time,point.params.frequency,point.params.inter_vehicle_distance,rho
count,378.000000,378.000000,378.000000,378.000000,378.000000
mean,0.384696,6.341652,5.803640,12.130109,2.888112
std,0.278122,2.199545,2.955983,5.380051,5.171567
min,0.001723,0.036584,0.057499,7.015904,-0.273221
25%,0.147265,6.013849,3.008545,8.997176,-0.210547
50%,0.327743,6.878283,5.602954,10.166939,0.247632
75%,0.600683,7.543055,8.890745,11.461895,3.119457
max,0.994191,9.980661,9.993043,29.065113,23.108299


In [20]:
print(len(evaluation_simulation_params_df) == len(evaluation_simulation_params_ce_df)+ len(evaluation_simulation_params_nce_df))

True


# Comparing Attributes Across sets
 Need to crate function for sepearting no crash, and attacker crash instances for analysis

In [ ]:
ce_ranges = {}
nce_ranges = {}
for i in intemittent_cols:
    ce_ranges[i] = np.max(evaluation_simulation_params_ce_df[i]) - np.min(evaluation_simulation_params_ce_df[i])
    nce_ranges[i] = np.max(evaluation_simulation_params_nce_df[i]) - np.min(evaluation_simulation_params_nce_df[i])
    print(np.max(evaluation_simulation_params_ce_df[i]), np.min(evaluation_simulation_params_ce_df[i]))

In [13]:
for i in intemittent_cols:
    print(f"Counter-example range for category {i}: {ce_ranges[i]}, non-counter example range: {nce_ranges[i]}")
    print(f"Category diff is: {np.abs(ce_ranges[i] - nce_ranges[i])}")

Counter-example range for category point.params.EGO_BRAKING_THRESHOLD: 9.753985421807162, non-counter example range: 9.980718524651893
Category diff is: 0.22673310284473125
Counter-example range for category point.params.inter_vehicle_distance: 4.00029747091369, non-counter example range: 22.961962636790023
Category diff is: 18.961665165876333
Counter-example range for category point.params.random_t1: 6.89047695759216, non-counter example range: 9.737961754091387
Category diff is: 2.847484796499227
Counter-example range for category point.params.random_t2: 6.718381098594222, non-counter example range: 8.87011135496638
Category diff is: 2.151730256372158
Counter-example range for category point.params.random_t3: 7.6661851936362915, non-counter example range: 8.91954048236823
Category diff is: 1.2533552887319388
Counter-example range for category point.params.random_t4: 8.817220946415041, non-counter example range: 8.857747223274195
Category diff is: 0.04052627685915411
Counter-example r

In [15]:
evaluation_simulation_params_nce_df.reset_index(drop=True, inplace=True)


 It seems that t1 attack, initial vehicle distance, and ego braking threshold are most significant in forcing a crash. These categories have the most range in between the two sets. (Counter-examples and non
)

In [16]:
examples = []

for i in range(0,len(evaluation_simulation_params_nce_df)):
    if np.abs(np.mean(evaluation_simulation_params_ce_df["point.params.inter_vehicle_distance"]) -
               evaluation_simulation_params_nce_df["point.params.inter_vehicle_distance"][i]) < .5:
      # print(evaluation_simulation_params_nce_df["point.params.inter_vehicle_distance"][i] - np.mean(evaluation_simulation_params_ce_df["point.params.inter_vehicle_distance"]))
        examples.append(evaluation_simulation_params_nce_df.iloc[i])

In [64]:
print(len(examples))

193


In [18]:
examples = pd.DataFrame(examples)

In [19]:
print(len(examples))

193


In [20]:
examples.describe()

,point.params.EGO_BRAKING_THRESHOLD,point.params.inter_vehicle_distance,point.params.random_t1,point.params.random_t2,point.params.random_t3,point.params.random_t4,point.params.random_thr_br1,rho
count,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000
mean,6.952226,9.744507,13.007916,26.327024,37.090970,48.978556,0.100100,-0.029217
std,1.861507,0.300492,1.243267,1.403338,0.978324,1.026812,0.089156,0.537032
min,5.014987,9.250714,10.042208,21.563227,31.199958,41.397373,0.000050,-0.281975
25%,5.623539,9.482965,12.348164,25.182132,36.730940,48.643257,0.044768,-0.240269
50%,6.285440,9.756275,12.964844,26.348285,37.195090,49.115231,0.084553,-0.220383
75%,6.960798,10.001100,13.607577,27.367353,37.689864,49.544296,0.138487,0.055326
max,12.950636,10.224737,19.096618,29.519233,39.115091,49.996488,0.860559,4.726166


In [21]:
evaluation_simulation_params_ce_df.describe()

,point.params.EGO_BRAKING_THRESHOLD,point.params.inter_vehicle_distance,point.params.random_t1,point.params.random_t2,point.params.random_t3,point.params.random_t4,point.params.random_thr_br1,rho
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,8.713029,9.746584,14.209324,27.643985,35.297768,46.427851,0.161618,-0.228502
std,2.963695,0.972200,2.646077,2.179032,2.071668,3.285723,0.201605,0.034751
min,5.106673,7.522011,10.461696,23.207072,31.539357,41.114363,0.000577,-0.252136
25%,6.104825,9.013499,11.627715,25.642673,34.005609,43.669777,0.041240,-0.245407
50%,8.886217,9.777100,14.556311,28.285041,35.536989,48.285184,0.100297,-0.240379
75%,10.510309,10.530265,16.743642,29.448682,36.425461,49.177634,0.189451,-0.235022
max,14.860658,11.522309,17.352173,29.925453,39.205542,49.931584,0.751167,-0.119553


# Collecting Extra Scenic data to construct new features for analysis

In [ ]:
def velocity_at_attack_intermittent(velocities_df, intermittent_df):
    simulation_velocities_arr = []
    velocities_df = velocities_df.reset_index(drop=True)
    
    for j in range(0,len(velocities_df)):
        sim_velocities = []
        for i in velocities_df.columns:
            vals = ast.literal_eval(velocities_df[i][j])
            velocities = [i[1] for i in vals]
            sim_velocities.append(velocities)
        
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance

    simulation_velocities_arr = np.array(simulation_velocities_arr)

    
    col_names = ["t1_velocity", "t2_velocity", "t3_velocity", "t4_velocity"]
    velocity_df = pd.DataFrame(simulation_velocities_arr, columns=col_names)

    return(pd.concat([intermittent_df, velocity_df],axis=1))

    

In [100]:
simulation_velocities_df = pd.read_csv("intermittent_attack.scenic1000_velocities.csv")
simulation_velocities_df = simulation_velocities_df.reset_index(drop=True)

In [ ]:
simulation_velocities_df

simulation_velocities_arr = []


for j in range(0,len(simulation_velocities_df)):
    sim_velocities = []
    for i in simulation_velocities_df.columns:
        vals = ast.literal_eval(simulation_velocities_df[i][j])
        velocities = [i[1] for i in vals]
        sim_velocities.append(velocities)
    
    simulation_velocities_arr.append(np.array(sim_velocities)) #accumulate a single simulation instance



# Gathering velocity at the time of attack

In [23]:
attack_times = evaluation_simulation_params_df[["point.params.random_t1", "point.params.random_t2","point.params.random_t3","point.params.random_t4"]]

KeyError: "None of [Index(['point.params.random_t1', 'point.params.random_t2',\n       'point.params.random_t3', 'point.params.random_t4'],\n      dtype='object')] are in the [columns]"

Velocity is measured at each timestep with is 0.1 or one tenth of a second. We need to account for this when we consider the velocity before the attacker slams on the breaks. 

In [ ]:
velocity_at_attack = []
for i in range(len(attack_times)):
    sim_attack = []
    for j in attack_times:
        idx = np.floor(attack_times[j][i]) * 10
        if idx > len(simulation_velocities_arr[i][0]): #check that the sim didn't end before the attack was instanciated
            sim_attack.append(0)
        else:
            sim_attack.append(simulation_velocities_arr[i][0][int(idx)]) 
    velocity_at_attack.append(np.array(sim_attack))


In [ ]:
len(velocity_at_attack)
for i in range(5):
    print(velocity_at_attack[i])

# More Stuff
